In [ ]:
!pip install gradio
!pip install elevenlabs
!pip install assemblyai

INFO: pip is looking at multiple versions of requests to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 18.1/18.1 MB 11.2 MB/s eta 0:00:00
   ---------------------------------------- 11.4/11.4 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.11
    Uninstalling urllib3-1.26.11:
      Successfully uninstalled urllib3-1.26.11
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.11.1
    Uninstalling tomlkit-0.11.1:
      Successfully uninstalled tomlkit-0.11.1
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
botocore 1.27.28 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.3.0 which is incompatible.
conda-repo-cli 1.0.27 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.27 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.27 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.


In [ ]:

import gradio as gr
import assemblyai as aai
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs
import uuid
from pathlib import Path

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

def voice_to_voice(audio_file,STABILITY,SIMILARITY,STYLE):
    
    #transcribe audio
    transcription_response = audio_transcription(audio_file)

    if transcription_response.status == aai.TranscriptStatus.error:
        raise gr.Error(transcription_response.error)
    else:
        text = transcription_response.text

    es_translation, tr_translation, ja_translation = text_translation(text)

    es_audi_path = text_to_speech(es_translation,STABILITY,SIMILARITY,STYLE)
    tr_audi_path = text_to_speech(tr_translation,STABILITY,SIMILARITY,STYLE)
    ja_audi_path = text_to_speech(ja_translation,STABILITY,SIMILARITY,STYLE)

    es_path = Path(es_audi_path)
    tr_path = Path(tr_audi_path)
    ja_path = Path(ja_audi_path)

    return es_path, tr_path, ja_path


def audio_transcription(audio_file):

    aai.settings.api_key = "c4fae7dfe3a6460b84e47b66fc12c530"

    transcriber = aai.Transcriber()
    transcription = transcriber.transcribe(audio_file)

    return transcription

def text_translation(text):
 
    
    es = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang='spa_Latn', max_length = 400)
    es_text = es(text)[0]['translation_text']

    tur = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang='tur_Latn', max_length = 400)
    tur_text = tur(text)[0]['translation_text']

    jp = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang='jpn_Jpan', max_length = 400)
    jp_text = jp(text)[0]['translation_text']
    
    return es_text, tur_text, jp_text

def text_to_speech(text,STAB,SIM,ST):


    client = ElevenLabs(
        api_key= "sk_02a3c1a9a18d279709f67fc9b786071920ffcdf42f03d5d4",
    )

    # Calling the text_to_speech conversion API with detailed parameters
    response = client.text_to_speech.convert(
        voice_id="2EiwWnXFnvU5JabPnv8n", #clone your voice on elevenlabs dashboard and copy the id
        optimize_streaming_latency="0",
        output_format="mp3_22050_32",
        text=text,
        model_id="eleven_multilingual_v2", # use the turbo model for low latency, for other languages use the `eleven_multilingual_v2`
        voice_settings=VoiceSettings(
            stability=STAB,
            similarity_boost=SIM,
            style=ST,
            use_speaker_boost=True,
        ),
    )

    # Generating a unique file name for the output MP3 file
    save_file_path = f"{uuid.uuid4()}.mp3"

    # Writing the audio to a file
    with open(save_file_path, "wb") as f:
        for chunk in response:
            if chunk:
                f.write(chunk)

    print(f"{save_file_path}: A new audio file was saved successfully!")

    # Return the path of the saved audio file
    return save_file_path

audio_input = gr.Audio(
    sources=["microphone"],
    type="filepath"
)

demo = gr.Interface(
    fn=voice_to_voice,
    inputs=[audio_input,gr.Slider(0,1),gr.Slider(0,1),gr.Slider(0,1)],
    outputs=[gr.Audio(label="Spanish"), gr.Audio(label="Turkish"), gr.Audio(label="Japanese")]
)

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(

42b61f30-1ec7-40a0-9c7a-8e1513550428.mp3: A new audio file was saved successfully!
1a81cdf0-d80a-415e-b11b-2f3f6314c8cc.mp3: A new audio file was saved successfully!
f7b188a5-0bb8-4ae4-96ba-e321a700beb6.mp3: A new audio file was saved successfully!


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(

ada22e5a-f46b-4f47-afec-a8dada4d03ff.mp3: A new audio file was saved successfully!
7ead2643-2d3d-4085-8094-0d5932f0f87d.mp3: A new audio file was saved successfully!
a5356b80-3cdb-4f86-9c7b-bed6801927b0.mp3: A new audio file was saved successfully!


In [9]:
!pip install --upgrade --upgrade-strategy eager optimum[onnxruntime]

   ---------------------------------------- 15.9/15.9 MB 11.4 MB/s eta 0:00:00
   ---------------------------------------- 11.1/11.1 MB 11.4 MB/s eta 0:00:00
   ---------------------------------------- 9.7/9.7 MB 11.6 MB/s eta 0:00:00
   ---------------------------------------- 14.5/14.5 MB 11.4 MB/s eta 0:00:00
   ---------------------------------------- 25.5/25.5 MB 11.3 MB/s eta 0:00:00
   ---------------------------------------- 2.4/2.4 MB 10.5 MB/s eta 0:00:00
   ---------------------------------------- 1.6/1.6 MB 9.8 MB/s eta 0:00:00
   ---------------------------------------- 11.6/11.6 MB 11.4 MB/s eta 0:00:00
   ---------------------------------------- 536.2/536.2 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2022.1
    Uninstalling pytz-2022.1:
      Successfully uninstalled pytz-2022.1
  Attempting uninstall: mpmath
    Found existing installation: mpmath 1.2.1
    Uninstalling mpmath-1.2.1:
      Successfully uninstalled mpmath-1.

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
botocore 1.27.28 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.3.0 which is incompatible.
conda-repo-cli 1.0.27 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.27 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.27 requires python-dateutil==2.8.2, but you have python-dateutil 2.9.0.post0 which is incompatible.
conda-repo-cli 1.0.27 requires PyYAML==6.0, but you have pyyaml 6.0.2 which

In [11]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer, pipeline


tokenizer = AutoTokenizer.from_pretrained("laiyer/xlm-roberta-base-language-detection-onnx")
model = ORTModelForSequenceClassification.from_pretrained("laiyer/xlm-roberta-base-language-detection-onnx")
classifier = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tokenizer,
    top_k=None,
)

classifier_output = ner("It's not toxic comment")
print(classifier_output)


ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "c:\Users\HP\anaconda3\New folder\python.exe"
  * The NumPy version is: "1.23.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The file cannot be accessed by the system.


RuntimeError: Failed to import optimum.onnxruntime.modeling_ort because of the following error (look up to see its traceback):


[{'translation_text': 'This Is a Very Good Model.'}]

Hola , ¿ cómo estás ?
